In [1]:
# bs4 for scraping, requests for requesting, pandas for data-ing
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# First website to be scraped
URL1 = "https://www.nonprofitnewyork.org/join/directory/"
page = requests.get(URL1)

In [3]:
# scrape relevant HTML
soup1 = BeautifulSoup(page.content, "html.parser")
all_non_profits = soup1.select('.tab-pane > ul > li')

In [4]:
# process scraped HTML and extract website + non profite name
nps = []

for np in all_non_profits:
    a = np.find("a")
    if a:
        nps.append({'website': a['href'], 'name': a.contents[0]})
    else:
        # text of the non linked nps is the second child of the li tag
        nps.append({'name': np.contents[1]})

In [8]:
# Save the data
df1 = pd.DataFrame(nps)
# print(df1.head())
df1.to_csv('1.csv')